In [1]:
import codecs
import glob
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm

In [20]:
data_dir = "data/input/励起状態/alpha1/1A/NH/S1/opt2/*.out"
result_file = data_dir.split('励起状態')[1].split('*')[0].replace('/', '_')[1:-1]
result_dir = "data/output/t2_" + result_file + ".csv"
paths = [p for p in glob.glob(data_dir)]


# Z-builderで見た時の番号
atom_number_include_dummy = [8, 9, 12, 13]
# ダミーを引いた時
atom_number = [n - 4 for n in atom_number_include_dummy]

torsion_2_key = ""
count = 1
# 二面角を示すkeywordの作成
for n in atom_number:
    if count == 1:
        torsion_2_key = str(n) + ","
        count += 1
    elif count < len(atom_number):
        torsion_2_key = torsion_2_key + str(n) + ","
        count += 1
    else:
        torsion_2_key = torsion_2_key + str(n)

initial_key = "Optimized Parameters"
end_key = "Grad"
df_output = pd.DataFrame()


In [21]:
for path in paths:
    # file_nameから角度の情報を取る
    file_name = path.split('/')[-1]
    torsion_angle_1 = int(file_name.split("@")[0].split("_")[-1])

    with codecs.open(path, "r", "UTF-8", "ignore") as file:
        df = pd.read_table(file)
        df = df.rename(columns={df.columns[0]: 'Col_1'})
        
        if df.iloc[-1].str.contains('Normal termination')['Col_1']:
            # initial_keyを含む行を探す
            initial_key_df = df[df['Col_1'].str.contains(initial_key, case=False)]
            initial_key_index = initial_key_df.index[0]
            df = df.iloc[initial_key_index:]
            # initial_keyを含む行以降のend_keyを含む行を抜き出す
            end_key_df = df[df['Col_1'].str.contains(end_key)]
    #         dfを更新しているのでここでの行番号は0から始まる
            end_key_index = end_key_df.index[0] - initial_key_index
            df = df.iloc[:end_key_index]
            # 該当行の抜き出しが完了

    #         二面角の抜き出し
            df = df[df['Col_1'].str.contains(torsion_2_key)]
            torsion_angle_2 = float(df['Col_1'].iloc[0].split(")")[1].split("-DE/DX")[0].replace(" ", ""))

            df_torsion_angle = pd.DataFrame(data=[torsion_angle_2], 
                                columns=["torsion2"],
                                index=[torsion_angle_1])
            df_output = pd.concat([df_output, df_torsion_angle])
        else:
            print(file_name)
            


In [22]:
df_output = df_output.sort_index()
df_output.to_csv(result_dir)
df_output

,torsion2
-165,-42.2023
-150,-68.5120
-30,174.6690
-15,164.1030
0,152.7200
15,138.2996
30,122.2420
150,-17.1933
165,-24.5501
180,-32.5915
